## Every Dream v2 Jupyter Notebook

### [General Instructions](https://github.com/victorchall/EveryDream2trainer/blob/main/README.md)

### Have you installed EveryDream2?
[Windows Setup](doc/SETUP.md)

[RunPod / Vast Setup](doc/CLOUD_SETUP.md)

### What's your plan?
You will want to have your data prepared before starting, and have a rough training plan in mind. 

**Make sure your images are captioned!**

By default the name of your image files are assumed to be captions. If you want to get fancy, there are [more sophisticated techniques](https://github.com/victorchall/EveryDream2trainer/blob/main/doc/DATA.md)

**If this is your first time trying a full fine-tune, start small!** 

Pick a single concept and 30-100 images, and see what happens. 

Training a small dataset like this is fast, and will give you a feel for how quickly your model (over-)trains depending on your training schedule, captioning schema, knob twiddling. This notebook provides some sensible defaults, there are more questions than answers in how best to fine tune anything. 

**_When_ you have questions...**

Come visit us at [EveryDream Discord](https://discord.gg/uheqxU6sXN)

# Upload training files

Use the navigation on the left to open the **/workspace/EveryDream2trainer/input** folder and upload your training files using the **up arrow button** above the file explorer, or by dragging and dropping the files from your local machine onto the file explorer.

If you have many training files, or nested folders of training data, create a zip archive of your training data, upload this file to the input folder, then right click on the zip file and select "Extract Archive".

### (Optional) Configure sample prompts
You can set your own sample prompts by adding them, one line at a time, to sample_prompts.txt.### (Optional) Weights and Biases login. 

### HuggingFace Login
Run the cell below and paste your token into the prompt.  You can get your token from your [huggingface account page](https://huggingface.co/settings/tokens).

The token will not show on the screen, just press enter after you paste it.

In [ ]:
from huggingface_hub import notebook_login, hf_hub_download
import os
notebook_login()

#### Then run the following cell to download the base checkpoint (may take a minute).

In [ ]:
repo="panopstor/EveryDream"
ckpt_file="sd_v1-5_vae.ckpt"

print(f"Downloading {ckpt_file} from {repo}")
downloaded_model_path = hf_hub_download(repo, ckpt_file, cache_dir="/workspace/hfcache")
ckpt_name = os.path.splitext(os.path.basename(downloaded_model_path))[0]
print(f"Downloaded {ckpt_name} to {downloaded_model_path}")

if not os.path.exists(f"ckpt_cache/{ckpt_name}"):
    print(f"Converting {ckpt_name} to Diffusers format")
    %run utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim \
    --original_config_file v1-inference.yaml \
    --image_size 512 \
    --checkpoint_path "{downloaded_model_path}" \
    --prediction_type epsilon \
    --upcast_attn False \
    --dump_path "ckpt_cache/{ckpt_name}"

print("DONE")

# Start Training
Naming your project will help you track what the heck you're doing when you're floating in checkpoint files later.

You may wish to consider adding "sd1" or "sd2v" or similar to remember what the base was, as you'll also have to tell your inference app what you were using, as its difficult for programs to know what inference YAML to use automatically. For instance, Automatic1111 webui requires you to copy the v2 inference YAML and rename it to match your checkpoint name so it knows how to load the file, tough it assumes SD 1.x compatible. Something to keep in mind if you start training on SD2.1.

`max_epochs`, `sample_steps`, and `save_every_n_epochs` should be tuned to your dataset. I like to generate one or two sets of samples per save, and aim for 5 (give or take 2) saved checkpoints.

Next cell runs training. This will take a while depending on your number of images, repeats, and max_epochs.

You can watch for test images in the logs folder.

## Weights and Balanaces
I you pass the `--wandb` flag you will be prompted for your W&B `API Key`. W&B is a free online logging utility. If you don't have a W&B account, you can create one for free at https://wandb.ai/site. Your key is on this page: https://wandb.ai/settings under "Danger Zone" "API Keys"

In [ ]:
%run train.py --config train.json \
--resume_ckpt "sd_v1-5_vae" \
--project_name "sd1_mymodel" \
--data_root "input" \
--max_epochs 200 \
--sample_steps 150 \
--save_every_n_epochs 35 \
--lr 1.2e-6 \
--lr_scheduler constant \
--save_full_precision


### Optionally you can chain trainings together using multiple configurations combined with `resume_ckpt: findlast`

In [ ]:
%run train.py --config chain0.json --project_name "sd1_chain_a" --data_root "input" --resume_ckpt "{ckpt_name}"
%run train.py --config chain1.json --project_name "sd1_chain_b" --data_root "input" --resume_ckpt findlast
%run train.py --config chain2.json --project_name "sd1_chain_c" --data_root "input" --resume_ckpt findlast

# HuggingFace upload
Use the cell below to upload one or more checkpoints to your personal HuggingFace account, if you want, instead of manually downloading. You should already be authorized to Huggingface by token if you used the download/token cells above.

* You can get your account name from your [HuggingFace account page](https://huggingface.co/settings/account). Look for your "username" field and paste it below.

* You only need to setup a repository one time.  You can create it here: [Create New HF Model](https://huggingface.co/new)  Make sure you write down the repo name you make for future use.  You can reuse it later.

In [ ]:
import glob
import os
from huggingface_hub import HfApi
from ipywidgets import *

all_ckpts = [f for f in glob.glob("*.ckpt")]
  
ckpt_picker = SelectMultiple(options=all_ckpts, layout=Layout(width="600px")) 
hfuser = Text(placeholder='Your HF user name')
hfrepo = Text(placeholder='Your HF repo name')

api = HfApi()
upload_btn = Button(description='Upload')
out = Output()

def upload_ckpts(_):
    repo_id=f"{hfuser.value}/{hfrepo.value}"
    with out:
        for ckpt in ckpt_picker.value:
            print(f"Uploading to HF: huggingface.co/{repo_id}/{ckpt}")
            response = api.upload_file(
                path_or_fileobj=ckpt,
                path_in_repo=ckpt,
                repo_id=repo_id,
                repo_type=None,
                create_pr=1,
            )
            display(response)
        print("DONE")
        print("Go to your repo and accept the PRs this created to see your files")

upload_btn.on_click(upload_ckpts)
box = VBox([ckpt_picker, HBox([hfuser, hfrepo]), upload_btn, out])

display(box)

# Test inference on your checkpoints

In [ ]:
from ipywidgets import *
from IPython.display import display, clear_output
import os
import gc
import random
import torch
import inspect

from torch import autocast
from diffusers import StableDiffusionPipeline, AutoencoderKL, UNet2DConditionModel, DDIMScheduler, DDPMScheduler, PNDMScheduler, EulerAncestralDiscreteScheduler
from transformers import CLIPTextModel, CLIPTokenizer


checkpoints_ts = []
for root, dirs, files in os.walk("."):
        for file in files:
            if os.path.basename(file) == "model_index.json":
                ts = os.path.getmtime(os.path.join(root,file))
                ckpt = root
                checkpoints_ts.append((ts, root))

checkpoints = [ckpt for (_, ckpt) in sorted(checkpoints_ts, reverse=True)]
full_width = Layout(width='600px')
half_width = Layout(width='300px')

checkpoint = Dropdown(options=checkpoints, description='Checkpoint:', layout=full_width)
prompt = Textarea(value='a photo of ', description='Prompt:', layout=full_width)
height = IntSlider(value=512, min=256, max=768, step=32, description='Height:', layout=half_width)
width = IntSlider(value=512, min=256, max=768, step=32, description='Width:', layout=half_width)
cfg = FloatSlider(value=7.0, min=0.0, max=14.0, step=0.2, description='CFG Scale:', layout=half_width)
steps = IntSlider(value=30, min=10, max=100, description='Steps:', layout=half_width)
seed = IntText(value=-1, description='Seed:', layout=half_width)
generate_btn = Button(description='Generate', layout=full_width)
out = Output()

def generate(_):
    with out:
        clear_output()
        display(f"Loading model {checkpoint.value}")
        actual_seed = seed.value if seed.value != -1 else random.randint(0, 2**30)

        text_encoder = CLIPTextModel.from_pretrained(checkpoint.value, subfolder="text_encoder")
        vae = AutoencoderKL.from_pretrained(checkpoint.value, subfolder="vae")
        unet = UNet2DConditionModel.from_pretrained(checkpoint.value, subfolder="unet")
        tokenizer = CLIPTokenizer.from_pretrained(checkpoint.value, subfolder="tokenizer", use_fast=False)
        scheduler = DDIMScheduler.from_pretrained(checkpoint.value, subfolder="scheduler")
        text_encoder.eval()
        vae.eval()
        unet.eval()

        text_encoder.to("cuda")
        vae.to("cuda")
        unet.to("cuda")

        pipe = StableDiffusionPipeline(
            vae=vae,
            text_encoder=text_encoder,
            tokenizer=tokenizer,
            unet=unet,
            scheduler=scheduler,
            safety_checker=None, # save vram
            requires_safety_checker=None, # avoid nag
            feature_extractor=None, # must be none of no safety checker
        )

        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
        
        print(inspect.cleandoc(f"""
              Prompt: {prompt.value}
              Resolution: {width.value}x{height.value}
              CFG: {cfg.value}
              Steps: {steps.value}
              Seed: {actual_seed}
              """))
        with autocast("cuda"):
            image = pipe(prompt.value, 
                generator=torch.Generator("cuda").manual_seed(actual_seed),
                num_inference_steps=steps.value, 
                guidance_scale=cfg.value,
                width=width.value,
                height=height.value
            ).images[0]
        del pipe
        gc.collect()
        with torch.cuda.device("cuda"):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
        display(image)
            
generate_btn.on_click(generate)
box = VBox(
    children=[
        checkpoint, prompt, 
        HBox([VBox([width, height]), VBox([steps, cfg])]), 
        seed, 
        generate_btn, 
        out]
)


display(box)